# Description

Ce notebook est utilisé pour requérir le calcul de la productivité de l’eau pour une saison personnalisée, un multiplicateur de la culture, et une zone des couches de la Productivité en Biomasse Brute de l’Eau (GBWP) de WaPOR ou la Productivité en Biomasse nette (NBWP) en utilisant l’API de WaPOR.

Vous aurez besoin de la clé de l’API de WaPOR pour utiliser ce notebook.

# Étape 1: Read APIToken

Lire la clé de l’API
Obtenir votre clé d’API à partir de https://wapor.apps.fao.org/profile. Entrez votre clé d’API en exécutant la cellule ci-dessous.


In [1]:
import requests
import json

path_query=r'https://io.apps.fao.org/gismgr/api/v1/query/'
path_sign_in=r'https://io.apps.fao.org/gismgr/api/v1/iam/sign-in/'

APIToken=input('Votre clé d’API : ')

Your API token: Enter Your API token


# Étape 2: Obtenir une clé d’autorisation d’accès (AccessToken)

Utilisation de la clé d’entrée d’API pour obtenir la clé d’autorisation d’accès

In [2]:
resp_signin=requests.post(path_sign_in,headers={'X-GISMGR-API-KEY':APIToken})
resp_signin = resp_signin.json()
AccessToken=resp_signin['response']['accessToken']
AccessToken

'eyJhbGciOiJSUzI1NiIsImtpZCI6IjIxODQ1OWJiYTE2NGJiN2I5MWMzMjhmODkxZjBiNTY1M2UzYjM4YmYiLCJ0eXAiOiJKV1QifQ.eyJuYW1lIjoiQmljaCBUcmFuIiwicGljdHVyZSI6Imh0dHBzOi8vbGg0Lmdvb2dsZXVzZXJjb250ZW50LmNvbS8tQkxvTDhOYUVLNm8vQUFBQUFBQUFBQUkvQUFBQUFBQUFCTDgvMlFJTzFRQUlxbW8vcGhvdG8uanBnIiwiaXNzIjoiaHR0cHM6Ly9zZWN1cmV0b2tlbi5nb29nbGUuY29tL3VuZmFvLWFwcHMtZGV2IiwiYXVkIjoidW5mYW8tYXBwcy1kZXYiLCJhdXRoX3RpbWUiOjE1OTQ4MTcyMDMsInVzZXJfaWQiOiJSNk5Ibkd5dDRUT3p1M212T2tteE04UkxFMnUxIiwic3ViIjoiUjZOSG5HeXQ0VE96dTNtdk9rbXhNOFJMRTJ1MSIsImlhdCI6MTU5NDgxNzIwMywiZXhwIjoxNTk0ODIwODAzLCJlbWFpbCI6ImJpY2h0bjk0QGdtYWlsLmNvbSIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJmaXJlYmFzZSI6eyJpZGVudGl0aWVzIjp7Imdvb2dsZS5jb20iOlsiMTAyMTY0MjA1NjAzMjU1NDMxMjQzIl0sImVtYWlsIjpbImJpY2h0bjk0QGdtYWlsLmNvbSJdfSwic2lnbl9pbl9wcm92aWRlciI6ImN1c3RvbSJ9fQ.w13Mg-FFLhxa2oyoEs_Rr0f7yXMc4ui4IxSoCBDWY10KSMoPasAH4_24MEO_ziEEyAbsIf2QGygI3Xvs-Dd3qa7H5XuVKZqR-IPcPaygNWu9rasuA-33n8X_iSEghDsuJocuJngD2DDpjXPzQ1oa6uneuXEztA8KilQYJ5ir0EoUKffFEGp5fBXUVmvMOmxN-dVYh-VPmETXpMjNd

# Étape 3: Étape la charge de la requête (QueryPayload)

**Arguments** 'waterProductivity' type, 'startSeason', 'endSeason', 'resolution','shape'

Lire [les notes explicatives sur la personnalisation de la productivité de l’eau](./data/explanatory-notes.pdf) pour savoir comment calculer le multiplicateur d’une culture (cropMultiplier)

In [3]:
waterProductivity="GBWP" #"GBWP" or "NBWP"
cropMultiplier=0.5
startSeason="2015-01" # "YYYY-DK" (Dekad)
endSeason= "2015-18" # "YYYY-DK" (Dekad)
resolution="100m" #"250m" or "100m" , maybe "30m" works for some area

Définir la forme du polygone de la zone d’intérêt

In [4]:
bbox= [37.95883206252312, 7.89534, 43.32093, 12.3873979377346] #latlon
xmin,ymin,xmax,ymax=bbox[0],bbox[1],bbox[2],bbox[3]
Polygon=[
          [xmin,ymin],
          [xmin,ymax],
          [xmax,ymax],
          [xmax,ymin],
          [xmin,ymin]
        ]

In [5]:
query ={
  "type": "CustomWaterProductivity",
  "params": {
    "waterProductivity": waterProductivity, #"GBWP" or "NBWP"
    "resolution": resolution, #"250m" ou "100m" , peut être "30m" marche pour certaines zones
    "startSeason": startSeason, # "YYYY-DK" (Dekad)
    "endSeason": endSeason, # "YYYY-DK" (Dekad),
    "cropMultiplier":cropMultiplier,
    "shape": {
      "type": "Polygon", #définir les coordonnées de la zone au format geojson
      "coordinates": [Polygon]
    }
  }
}

# Étape 4: Poster la charge de la requête (QueryPayload) avec la clé d’accès (AccessToken) dans l’entête (Header)

En réponses, obtient un url pour requérir le travail

In [6]:
resp_query=requests.post(path_query,headers={'Authorization':'Bearer {0}'.format(AccessToken)},
                                          json=query)
resp_query = resp_query.json()
job_url=resp_query['response']['links'][0]['href']

job_url

'https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/jobs/e32877e8-1431-40e2-ac5e-badd445b8e8d'

# Étape 5: Obtenir les résultats du travail (Job)

Cela mettra du temps avant que le travail ne soit fini. Une fois le travail fini, son statut changera de « En cours » (RUNNING) à « Terminé » (COMPLETED) ou « terminé avec erreurs » (COMPLETED WITH ERRORS). Si c’est terminé, les résultats de la série chronologique de la zone peuvent être trouvés à partir de Réponse « sortie ».

In [8]:
i=0
print('RUNNING',end=" ")
while i==0:        
    resp = requests.get(job_url)
    resp=resp.json()
    
    if resp['response']['status']=='RUNNING':
        print('.',end =" ")
    if resp['response']['status']=='COMPLETED':
        results=resp['response']['output']
        print('Link to download GBWP',results['bwpDownloadUrl'])
        print('Link to download TBP',results['tbpDownloadUrl'])
        print('Link to download AETI',results['wtrDownloadUrl'])
        i=1
    if resp['response']['status']=='COMPLETED WITH ERRORS':
        print(resp['response']['log'])
        i=1       

RUNNING Link to download GBWP https://io.apps.fao.org/gismgr/download/1ed99653-21e6-4441-b207-7ec9f1cf41a9/L2_GBWP_1501-1518.tif
Link to download TBP https://io.apps.fao.org/gismgr/download/1ed99653-21e6-4441-b207-7ec9f1cf41a9/L2_TBP_1501-1518.tif
Link to download AETI https://io.apps.fao.org/gismgr/download/1ed99653-21e6-4441-b207-7ec9f1cf41a9/L2_AETI_1501-1518.tif
